## 简介

### Proxy

Proxy 可以用来代理一个对象，拦截对象的 [internal method](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Proxy#object_internal_methods)。

他的构造函数接受一个 target对象 和一个 handler对象。target对象就是需要被代理的对象，而 hanlder对象 里是一系列的方法，对应 JS 的对象的 internal method。

关于什么操作会触发什么样的 internal method，可以查看[ECMA-262 标准](https://ecma-international.org/publications-and-standards/standards/ecma-262/)。

比如，可以看出 get 的时候会触发 `[[Get]]` 方法
![GetMthod](./pics/GetMthod.png)

handler 所有方法的列表：


| `handler` 方法         | Internal method        | 触发 Internal method 的常用操作                            |
|-----------------------|------------------------|----------------------------------------------------------|
| `getPrototypeOf`      | [[GetPrototypeOf]]     | `Object.getPrototypeOf(proxy)`, `proxy.__proto__`         |
| `setPrototypeOf`      | [[SetPrototypeOf]]     | `Object.setPrototypeOf(proxy, proto)`                      |
| `isExtensible`        | [[IsExtensible]]       | `Object.isExtensible(proxy)`                               |
| `preventExtensions`   | [[PreventExtensions]]  | `Object.preventExtensions(proxy)`                          |
| `getOwnPropertyDescriptor` | [[GetOwnProperty]] | `Object.getOwnPropertyDescriptor(proxy, prop)`            |
| `defineProperty`      | [[DefineOwnProperty]]  | `Object.defineProperty(proxy, prop, descriptor)`          |
| `has`                 | [[HasProperty]]        | `key in obj`, `Reflect.has(proxy, prop)`               |
| `get`                 | [[Get]]                | `proxy[prop]`, `Reflect.get(proxy, prop)`                 |
| `set`                 | [[Set]]                | `proxy[prop] = value`, `Reflect.set(proxy, prop, value)`  |
| `deleteProperty`      | [[Delete]]             | `delete proxy[prop]`, `Reflect.deleteProperty(proxy, prop)`|
| `apply`               | [[Call]]               | `proxy()`, `Reflect.apply(proxy, thisArg, args)`          |
| `construct`           | [[Construct]]          | `new proxy()`, `Reflect.construct(proxy, args, newTarget)`|
| `ownKeys`             | [[OwnPropertyKeys]]    |  `Object.keys()`, `Reflect.ownKeys()`                     |

### Reflect

Reflect 是一个不可构造的对象，内置了和 Proxy handler 一一对应的静态方法。所以经常结合 Proxy 一起使用。

可以简单的理解为把对 把对 receiver 调用的方法反射到 target 对象上。

在写 handler 逻辑的时候，比如我们需要拦截 target 的 get 方法，对于某些不存在的 get 不是返回 undefined，而是返回一个自定义的信息。

In [21]:
const john = {
  name: 'John',
  age: '18',
};

const proxyJohn = new Proxy(john, {
  get(target, key, receiver) {
    if (key in target) {
      return Reflect.get(target, key, receiver);
    }
    return `unknown information (${key.toString()}) about john`;
  }
});

通过 john 和 proxyJohn 都可以拿到 name 属性。而尝试获取 gender 属性的时候，直接对 target 对象 get gender，会直接返回一个 undefined，而通过 proxyJohn get gender 属性，则会提示 "unknown information gender of john"。

所以在这里我们拦截的 target 对象的 get 方法，可以对 object 的 internal method 做一些自定义行为。

In [ ]:
console.log(proxyJohn.name); // John
console.log(john.name) // John

console.log(proxyJohn.gender); // unknown information gender of john
console.log(john.gender); // undefined

## 用 Proxy 和 Reflect 可以做什么

在前端最出名的肯定是 Vue3 的响应式数据的底层原理。

通过 Proxy 拦截数据的 set 操作，在 set 之后插入渲染函数的执行，之后在对数据进行修改的时候，就会自动的调用副作用的渲染函数，达到响应式数据的效果。

In [31]:
const data = {
  value: "initial val"
};

const proxyData = new Proxy(data, {
  get(target, key, receiver) {
    return Reflect.get(target, key, receiver);
  },
  set(target, key, receiver) {
    Reflect.set(target, key, receiver);
    renderWithSideEffect();
    return true;
  }
});

function renderWithSideEffect() {
  for (const [key, val] of Object.entries(data)) {
    console.log(`key: ${key}, val: ${val}`);
  }
}

In [ ]:
proxyData.value = "set data";

// key: value, val: set data

In [ ]:
proxyData["count"] = 1;

// key: value, val: set data
// key: count, val: 1